# Preprocessing SODA and Training GNNs

by Ding

## 1. Preprocessing

In [1]:
!pip install geopandas

import numpy as np
from netCDF4 import Dataset
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point

from google.colab import drive
drive.mount("/gdrive", force_remount=True)

!cp -a "/gdrive/MyDrive/soda_331_pt_l5.nc" "/content/"

     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 48.2 MB/s 
     |████████████████████████████████| 16.7 MB 51.5 MB/s 
Mounted at /gdrive


In [2]:
soda = xr.open_dataset("soda_331_pt_l5.nc", decode_times=False)

soda_array = soda.to_array(dim="temp")
soda_smaller = soda_array[:,:,:,::5,::5].to_dataset(dim="temp")

start_year = 1980
end_year = 2009
target_year = end_year + +1
start_month = (start_year - 1980) * 12
end_month = (end_year + 1 - 1980) * 12

soda_sst_1980_2009 = np.zeros((end_month-start_month+12,1,66,144)) # Include one more year.
soda_sst_1980_2009[:,:,:,:] = soda_smaller.variables["temp"][0:end_month-start_month+12,:,:,:]
soda_sst_1980_2009 = np.squeeze(soda_sst_1980_2009, axis=1)
print(soda_sst_1980_2009.shape)

soda_sst_2010 = np.zeros((12,1,66,144))
soda_sst_2010[:,:,:,:] = soda_smaller.variables["temp"][end_month-start_month:end_month-start_month+12,:,:,:]
soda_sst_2010 = np.squeeze(soda_sst_2010, axis=1)
print(soda_sst_2010.shape)

(372, 66, 144)
(12, 66, 144)


In [3]:
soda_sst_1980_2009_transposed = soda_sst_1980_2009.transpose(1,2,0)
soda_sst_1980_2009_flattened = soda_sst_1980_2009_transposed.reshape(soda_sst_1980_2009.shape[1] * soda_sst_1980_2009.shape[2],len(soda_sst_1980_2009))
print(soda_sst_1980_2009_flattened.shape)

soda_sst_2010_transposed = soda_sst_2010.transpose(1,2,0)
soda_sst_2010_flattened = soda_sst_2010_transposed.reshape(soda_sst_2010.shape[1] * soda_sst_2010.shape[2],len(soda_sst_2010))
print(soda_sst_2010_flattened.shape)

(9504, 372)
(9504, 12)


In [4]:
def dropna(arr, *args, **kwarg):
    assert isinstance(arr, np.ndarray)
    dropped=pd.DataFrame(arr).dropna(*args, **kwarg).values
    if arr.ndim==1:
        dropped=dropped.flatten()
    return dropped

soda_sst_ocean_1980_2009_flattened = dropna(soda_sst_1980_2009_flattened)
print(soda_sst_ocean_1980_2009_flattened.shape)

soda_sst_ocean_2010_flattened = dropna(soda_sst_2010_flattened)
print(soda_sst_ocean_2010_flattened.shape)

(6924, 372)
(6924, 12)


In [5]:
target_month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
#target_month = ["Jan"]

soda_sst_ocean_1980_2009_flattened_all = []
soda_sst_ocean_2010_mhw_all = []

for month in target_month:

  index = target_month.index(month)
  soda_sst_ocean_1980_2009_flattened_all.append(soda_sst_ocean_1980_2009_flattened[:,index:(index+soda_sst_ocean_1980_2009_flattened.shape[1]-12)])

  soda_sst_ocean_2010_mhw = []
  for row in range(len(soda_sst_ocean_1980_2009_flattened)):
    sst_node = soda_sst_ocean_1980_2009_flattened[row][index::12]
    #print(np.mean(sst_node))
    if soda_sst_ocean_2010_flattened[row][index] > sorted(sst_node)[-4]:
      soda_sst_ocean_2010_mhw.append(1)
    else:
      soda_sst_ocean_2010_mhw.append(0)
  
  print(month, target_year, ":", soda_sst_ocean_2010_mhw.count(1), "MHW")
  print(month, target_year, ":", soda_sst_ocean_2010_mhw.count(0), "no MHW")
  print()

  soda_sst_ocean_2010_mhw = np.array(soda_sst_ocean_2010_mhw)
  soda_sst_ocean_2010_mhw_all.append(soda_sst_ocean_2010_mhw)

Jan 2010 : 1318 MHW
Jan 2010 : 5606 no MHW

Feb 2010 : 1427 MHW
Feb 2010 : 5497 no MHW

Mar 2010 : 1542 MHW
Mar 2010 : 5382 no MHW

Apr 2010 : 1440 MHW
Apr 2010 : 5484 no MHW

May 2010 : 1370 MHW
May 2010 : 5554 no MHW

Jun 2010 : 1363 MHW
Jun 2010 : 5561 no MHW

Jul 2010 : 1283 MHW
Jul 2010 : 5641 no MHW

Aug 2010 : 1444 MHW
Aug 2010 : 5480 no MHW

Sep 2010 : 1413 MHW
Sep 2010 : 5511 no MHW

Oct 2010 : 1368 MHW
Oct 2010 : 5556 no MHW

Nov 2010 : 1251 MHW
Nov 2010 : 5673 no MHW

Dec 2010 : 1367 MHW
Dec 2010 : 5557 no MHW



In [11]:
lons, lats = np.meshgrid(soda_smaller.longitude.values, soda_smaller.latitude.values)

soda_time_1 = soda_smaller.temp.isel(depth=0,time=240)
soda_time_1_lons, soda_time_1_lats = np.meshgrid(soda_time_1.longitude.values, soda_time_1.latitude.values)
soda_masked = soda_time_1.where(abs(soda_time_1_lons) + abs(soda_time_1_lats) > 0)

soda_masked.values.flatten()[soda_masked.notnull().values.flatten()]

from sklearn.metrics.pairwise import haversine_distances

lons_ocean = soda_time_1_lons.flatten()[soda_masked.notnull().values.flatten()]
lons_ocean = lons_ocean[::]
lats_ocean = soda_time_1_lats.flatten()[soda_masked.notnull().values.flatten()]
lats_ocean = lats_ocean[::]

lons_ocean *= np.pi/180
lats_ocean *= np.pi/180

points_ocean = np.concatenate([np.expand_dims(lats_ocean.flatten(),-1), np.expand_dims(lons_ocean.flatten(),-1)],-1)

distance_ocean = 6371*haversine_distances(points_ocean)

distance_ocean_diag = distance_ocean
distance_ocean_diag[distance_ocean_diag==0] = 1

distance_ocean_recip = np.reciprocal(distance_ocean_diag)

distance_ocean_recip.shape

(6924, 6924)

In [18]:
adjacency_matrix = distance_ocean_recip

w = []

for i in range(distance_ocean_recip.shape[0]):
  for j in range(distance_ocean_recip.shape[1]):
    if i != j:
      w.append(distance_ocean_recip[i][j])

weights = torch.tensor(w)

## 2. GNN

In [6]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

import random

import torch

import dgl
import dgl.nn as dglnn
import dgl.data
import torch.nn as nn
import torch.nn.functional as F

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.1 MB 2.7 MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [7]:
node_num = len(soda_sst_ocean_1980_2009_flattened_all[index])

def graph_structure():
  u = []
  v = []
  for i in range(node_num):
    for j in range(node_num):
      if j == i:
        pass
      else:
        u.append(i)
        v.append(j)
  return torch.tensor(u), torch.tensor(v)
  
u, v = graph_structure()
graph = dgl.graph((u, v))

In [8]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [9]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # Inputs are features of nodes.
        h = self.conv1(graph, inputs)
        h = torch.tanh(h) # sigmoid(h), relu(h)
        h = self.conv2(graph, h)
        return h

def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        #print("Validation / test observed class:", labels.tolist(), "| predicted class:", indices.tolist())
        #correct = torch.sum(indices == labels)
        combine = indices + labels
        accuracy = torch.sum(indices == labels).item() * 1.0 / len(labels)
        precision = (torch.sum(combine == 2).item() * 1.0) / (torch.sum(labels == 1).item() * 1.0)
        return accuracy, precision

In [ ]:
test_results = []

for month in target_month:
  
  print("--------------------")
  print("Start training for", month, ".")
  print()

  index = target_month.index(month)
  node_num = len(soda_sst_ocean_1980_2009_flattened_all[index])

  node_feature = torch.tensor(soda_sst_ocean_1980_2009_flattened_all[index])
  node_class = torch.tensor(soda_sst_ocean_2010_mhw_all[index])

  graph.ndata["feat"] = node_feature
  graph.ndata["label"] = node_class

  graph.edata['w'] = weights

  def masks():
    train = [True] * node_num
    val = [False] * node_num
    test = [False] * node_num
    randomlist = []
    for i in range(0, round(node_num * 0.3)):
      n = random.randint(0, node_num-1) # Need to substract 1 to be in the list index range.
      randomlist.append(n)
    #print(randomlist)
    randomlist_split = list(split(randomlist, 3))
    for i in randomlist_split[0]:
      val[i] = True
    for i in randomlist_split[1]:
      test[i] = True
    for i in randomlist_split[2]:
      test[i] = True
    for i in randomlist:
      train[i] = False
    #print("Training mask:")
    #print(train)
    return torch.tensor(train), torch.tensor(val), torch.tensor(test)

  graph.ndata["train_mask"], graph.ndata["val_mask"], graph.ndata["test_mask"] = masks()
  #print(graph.ndata)

  node_features = graph.ndata['feat']
  node_labels = graph.ndata['label']
  train_mask = graph.ndata['train_mask']
  valid_mask = graph.ndata['val_mask']
  test_mask = graph.ndata['test_mask']
  n_features = node_features.shape[1]
  n_labels = int(node_labels.max().item() + 1)

  model = SAGE(in_feats=n_features, hid_feats=200, out_feats=n_labels)
  #opt = torch.optim.Adam(model.parameters(), lr=0.0005)
  opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.99)

  for epoch in range(5):
      print("Epoch:", epoch+1)
      model.train()
      # Forward propagation by using all nodes
      logits = model(graph, node_features.float())
      # Compute loss.
      loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
      # Compute validation accuracy.
      acc, pre = evaluate(model, graph, node_features.float(), node_labels, valid_mask)
      # Backward propagation
      opt.zero_grad()
      loss.backward()
      opt.step()
      print("Training loss: %.4f" % loss.item(), "| validation accuracy: %.4f" % acc, "| validation precision: %.4f" % pre)
      print()

  print("----------")
  test_acc, test_pre = evaluate(model, graph, node_features.float(), node_labels, test_mask)
  print("Test accuracy: %.4f" % test_acc, "| test precision: %.4f" % test_pre)
  
  test_results.append([test_acc, test_pre])
  print("The test results for", month, "have been appended.")
  print()

--------------------
Start training for Jan .

Epoch: 1
Training loss: 0.8742 | validation accuracy: 0.6621 | validation precision: 0.3429

Epoch: 2
Training loss: 0.8573 | validation accuracy: 0.6621 | validation precision: 0.3429

Epoch: 3
Training loss: 0.8249 | validation accuracy: 0.6636 | validation precision: 0.3238

Epoch: 4
Training loss: 0.7767 | validation accuracy: 0.6713 | validation precision: 0.3238

Epoch: 5
Training loss: 0.7148 | validation accuracy: 0.6820 | validation precision: 0.3238

----------
Test accuracy: 0.6496 | test precision: 0.2623
The test results for Jan have been appended.

--------------------
Start training for Feb .

Epoch: 1
Training loss: 3.5350 | validation accuracy: 0.2033 | validation precision: 1.0000

Epoch: 2
Training loss: 3.1501 | validation accuracy: 0.2033 | validation precision: 1.0000

Epoch: 3
Training loss: 2.2418 | validation accuracy: 0.2048 | validation precision: 1.0000

Epoch: 4
Training loss: 1.6604 | validation accuracy: 0.22

In [ ]:
for element in test_results:
  print(element)

[0.6495589414595028, 0.26229508196721313]
[0.47219983883964545, 0.6775510204081633]
[0.7680817610062893, 0.010380622837370242]
[0.49801113762927607, 0.24436090225563908]
[0.2752808988764045, 0.44907407407407407]
[0.7859424920127795, 0.05803571428571429]
[0.7495987158908507, 0.07296137339055794]
[0.7797947908445146, 0.01098901098901099]
[0.7599364069952306, 0.10887096774193548]
[0.8247588424437299, 0.0]
[0.750402576489533, 0.06779661016949153]
[0.8076298701298701, 0.004310344827586207]
